In [2]:
import json
import subprocess
import pandas as pd
from pathlib import Path
import prody as pr

smiles = "COC1=CC=C(N2C3=C(C(C(N)=O)=N2)CCN(C4=CC=C(N5CCCCC5=O)C=C4)C3=O)C=C1"


def get_boltz_yaml_boilerplate(sequence: str, smiles: str, predict_affinity: bool):
    boltz_yaml = f"version: 1\nsequences:\n  - protein:\n      id: A\n      sequence: {sequence}\n      msa: empty\n  - ligand:\n      id: B\n      smiles: '{smiles}'\n"
    if predict_affinity:
        boltz_yaml += "properties:\n  - affinity:\n      binder: B\n"
    return boltz_yaml


def get_boltz_results_metadata(pdb_path: Path):
    protein = pr.parsePDB(str(pdb_path))
    sequence = protein.ca.getSequence()

    protein_plddt = protein.select('chid A and name CA').getBetas().mean() / 100
    ligand_plddt = protein.select('chid B and not element H').getBetas().mean() / 100
    affinity_meta_path = pdb_path.parent / f'affinity_{pdb_path.stem.replace("_model_0", "")}.json'
    confidence_meta_path = pdb_path.parent / f'confidence_{pdb_path.stem}.json'

    ptm, iptm, pbind, paffinity = None, None, None, None

    with affinity_meta_path.open('r') as f:
        affinity_meta = json.load(f)
        pbind = affinity_meta['affinity_probability_binary']
        paffinity = affinity_meta['affinity_pred_value']

    with confidence_meta_path.open('r') as f:
        confidence_meta = json.load(f)
        iptm = confidence_meta['iptm']
        ptm = confidence_meta['ptm']
    
    return {
        'seq': sequence, 'iptm': iptm, 'pbind': pbind, 
        'paffinity': paffinity, 'ligand_plddt': ligand_plddt, 'ptm': ptm, 
        'boltz_path': pdb_path, 'protein_plddt': protein_plddt
    }


def nise_round(input_dir: Path):

    input_paths = []
    for path in sorted(input_dir.glob('*.pdb')):
        input_paths.append(path)
    
    # Protonate input paths.
    for path in input_paths:
        subprocess.run(f'/nfs/polizzi/bfry/programs/NISE/protonate_and_add_conect_records.py {path.absolute()} "{smiles}" {path.absolute()}', shell=True)

    # Create inputs.txt file
    nise_paths = Path('nise_input_paths.txt')
    with nise_paths.open('w') as f:
        for path in input_paths:
            f.write(f'{path.absolute()}\n')

    # Run laser.
    laser_odir = input_dir.parent / f'{input_dir.stem}_laser'
    subprocess.run(f'cd /nfs/polizzi/bfry/programs; python -m LASErMPNN.run_batch_inference {nise_paths.absolute()} {laser_odir.absolute()} 5 -d cuda:0', shell=True)

    boltz_inputs_dir = laser_odir.parent / f'{laser_odir.stem}_boltz_inputs'
    boltz_inputs_dir.mkdir(exist_ok=True)
    for idx, path in enumerate(laser_odir.glob('*/*.pdb')):
        protein = pr.parsePDB(str(path))
        seq = protein.ca.getSequence()
        odir = boltz_inputs_dir / f'seq_{idx:04d}.yaml'
        with odir.open('w') as f:
            f.write(get_boltz_yaml_boilerplate(seq, smiles, predict_affinity=True))

    # Run Boltz.
    device_str='CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7'
    subprocess.run(f'{device_str} /nfs/polizzi/bfry/miniforge3/envs/boltz2_retry/bin/boltz predict --output_format pdb --devices 8 --use_potentials {boltz_inputs_dir.absolute()}', shell=True)

    all_boltz_results = []
    boltz_outputs_path = boltz_inputs_dir.parent / f"boltz_results_{boltz_inputs_dir.stem}"
    for path in boltz_outputs_path.glob('predictions/*/*.pdb'):
        all_boltz_results.append(get_boltz_results_metadata(path))
    
    return pd.DataFrame(all_boltz_results)
    
    
df = nise_round(Path('/nfs/polizzi/bfry/programs/CARPdock/apixaban_ntf2_screen/selected_poses'))

[09:53:56] WARNING: More than one matching pattern found - picking one

[09:53:57] WARNING: More than one matching pattern found - picking one

[09:53:58] WARNING: More than one matching pattern found - picking one

[09:53:59] WARNING: More than one matching pattern found - picking one

[09:54:00] WARNING: More than one matching pattern found - picking one

[09:54:01] WARNING: More than one matching pattern found - picking one

[09:54:01] WARNING: More than one matching pattern found - picking one

[09:54:02] WARNING: More than one matching pattern found - picking one

[09:54:03] WARNING: More than one matching pattern found - picking one

[09:54:04] WARNING: More than one matching pattern found - picking one

[09:54:05] WARNING: More than one matching pattern found - picking one

[09:54:05] WARNING: More than one matching pattern found - picking one

[09:54:06] WARNING: More than one matching pattern found - picking one

[09:54:07] WARNING: More than one matching pattern found - picki

[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK
Processing /nfs/polizzi/bfry/programs/CARPdock/apixaban_ntf2_screen/nise_input_paths.txt:
Generating 5 designs for 269 inputs with /nfs/polizzi/bfry/programs/LASErMPNN/model_weights/laser_weights_0p1A_nothing_heldout.pt on cuda:0 at temperature None


100%|██████████| 54/54 [03:12<00:00,  3.57s/it]                           


Checking input data.
Processing 1345 inputs with 512 threads.


  0%|          | 0/1345 [00:00<?, ?it/s]

...

[rank6]:[W930 10:43:59.956255130 TCPStore.cpp:125] [c10d] recvValue failed on SocketImpl(fd=130, addr=[localhost]:42164, remote=[localhost]:35591): Failed to recv, got 0 bytes. Connection was likely closed. Did the remote server shutdown or crash?
Exception raised from recvBytes at /pytorch/torch/csrc/distributed/c10d/Utils.hpp:682 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x80 (0x7fca1948eeb0 in /nfs/polizzi/bfry/miniforge3/envs/boltz2_retry/lib/python3.11/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x5d694d1 (0x7fca5af104d1 in /nfs/polizzi/bfry/miniforge3/envs/boltz2_retry/lib/python3.11/site-packages/torch/lib/libtorch_cpu.so)
frame #2: <unknown function> + 0x5d6a8cd (0x7fca5af118cd in /nfs/polizzi/bfry/miniforge3/envs/boltz2_retry/lib/python3.11/site-packages/torch/lib/libtorch_cpu.so)
frame #3: <unknown function> + 0x5d6b47a (0x7fca5af1247a in /

Number of failed examples: 0

Predicting property: affinity

Checking input data for affinity.
Running affinity prediction for 1345 inputs.


You are using a CUDA device ('NVIDIA H200') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H200') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H200') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_m

Predicting DataLoader 0:  98%|█████████▊| 166/169 [18:03<00:19,  0.15it/s]Number of failed examples: 0
Number of failed examples: 0
Number of failed examples: 0
Predicting DataLoader 0:  99%|█████████▉| 167/169 [18:10<00:13,  0.15it/s]Number of failed examples: 0
Number of failed examples: 0
Predicting DataLoader 0: 100%|██████████| 169/169 [18:23<00:00,  0.15it/s]
Number of failed examples: 0


[rank1]:[W930 11:02:54.177606931 TCPStore.cpp:125] [c10d] recvValue failed on SocketImpl(fd=136, addr=[localhost]:49122, remote=[localhost]:35591): Failed to recv, got 0 bytes. Connection was likely closed. Did the remote server shutdown or crash?
Exception raised from recvBytes at /pytorch/torch/csrc/distributed/c10d/Utils.hpp:682 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x80 (0x7fca1948eeb0 in /nfs/polizzi/bfry/miniforge3/envs/boltz2_retry/lib/python3.11/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x5d694d1 (0x7fca5af104d1 in /nfs/polizzi/bfry/miniforge3/envs/boltz2_retry/lib/python3.11/site-packages/torch/lib/libtorch_cpu.so)
frame #2: <unknown function> + 0x5d6a8cd (0x7fca5af118cd in /nfs/polizzi/bfry/miniforge3/envs/boltz2_retry/lib/python3.11/site-packages/torch/lib/libtorch_cpu.so)
frame #3: <unknown function> + 0x5d6b47a (0x7fca5af1247a in /

In [3]:
df = df.sort_values('pbind', ascending=False).copy()

In [4]:
df['score'] = df.ligand_plddt + df.pbind
df = df.sort_values('score', ascending=False).copy()

In [5]:
df

seq      iptm     pbind  \
943   APSLEEQCIALGREFVAAANAGDVEAMRSMLAPDATMTLSDGTKLT...  0.982275  0.990359   
1288  APSLRDQAIALGRAFVAAANAGDVEAMLGMLHPDATMTLSDGTKLT...  0.977896  0.990412   
374   APSLEEQCRQLGEAFVAAANAGDVEAMLSMLAPDATMTLSDGTKLT...  0.980509  0.975274   
938   PTDAEQEAICNQYAQALNDGDIDALLALFAPDAVVTMSDGTVLTSL...  0.983202  0.959201   
762   MTKEEMVKRAKEMVHEFFDAVNAGDEEKVREILADDFKMVDETGKE...  0.974145  0.974055   
...                                                 ...       ...       ...   
720   ATAAEMRTKARELVKAVEAGDLEAIRALAKPDAKAVLVSTGAKYNS...  0.668447  0.251654   
430   MSKEEIKAHIEKLLKAIASGDIDALKALLDPNIQALNKTKNKKYNT...  0.680719  0.216107   
1037  SAEEAIKALVTAFFDAMVAGDLDKVRSLAHPDVTGTSTSGGVANSL...  0.667437  0.238718   
1018  ASEEAIQAAEKAVKEGYAAGNNKDVEAIASFFTDDTVMTANHLNKT...  0.764848  0.190010   
416   MSDEAAKAEVRRICAQLIAAFETGDEATLREVLADDFVGVDLDTGK...  0.360300  0.247671   

      paffinity  ligand_plddt       ptm  \
943   -2.001191      0.978159  0.977154   
1288  -1.960841      0.974903  0.972152   
374   -1.368702      0.983538  0.975120   
938   -1.399056      0.971035  0.978275   
762   -1.316855      0.952982  0.965363   
...         ...           ...       ...   
720    0.490647      0.371959  0.621148   
430    0.791630      0.392144  0.776699   
1037   0.594988      0.365682  0.789892   
1018   0.336885      0.408735  0.880100   
416    1.003805      0.166121  0.390447   

                                             boltz_path  protein_plddt  \
943   /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.979065   
1288  /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.975720   
374   /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.982659   
938   /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.978046   
762   /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.959300   
...                                                 ...            ...   
720   /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.640817   
430   /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.723804   
1037  /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.816141   
1018  /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.707440   
416   /nfs/polizzi/bfry/programs/CARPdock/apixaban_n...       0.432785   

         score  
943   1.968518  
1288  1.965315  
374   1.958813  
938   1.930237  
762   1.927038  
...        ...  
720   0.623613  
430   0.608251  
1037  0.604401  
1018  0.598745  
416   0.413791  

[1345 rows x 9 columns]

In [9]:
pd.to_pickle(df, 'nise_round1_apixaban_ntf2.pkl')

In [11]:
import shutil

opath = Path('./nise_round1_apixaban_ntf2_top20/')
opath.mkdir(exist_ok=True)

for i in range(20):
    # print(df.iloc[0].boltz_path.absolute())
    shutil.copy(df.iloc[i].boltz_path.absolute(), opath / f'{df.iloc[i].boltz_path.stem}_rank_{i+1:02d}.pdb')

In [57]:
import shutil
top_by_score = Path('./top_by_score/')
top_by_score.mkdir(exist_ok=True)

for i, row in df.head(25).iterrows():
    shutil.copy(row.boltz_path, top_by_score)